# Make Introduction figure

Plots both morphologies and shows the regions.

Add placeholder for cable illustration.

# Imports

In [ ]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
from PIL import Image

import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '..', 'pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils
import plot_utils

In [ ]:
plot_utils.set_rcParams()

In [ ]:
fig_num = os.getcwd().split('/')[-1][3:5]
print(fig_num)

# Retsim color mapping

In [ ]:
with open('retsim_colors.inc', 'r') as f:
    lines = f.readlines()

colors = [
    'BlueViolet',
    'NeonBlue',
    'NewMidnightBlue',
    'SummerSky',
    'Red',
    'Black',
    'Gray40',
    'MediumBlue',
    'Navy',
    'Orange',
]

colors2rgb = {}
for color in colors:
    if color == 'Gray40':
        colors2rgb[color] = (0.4, 0.4, 0.4)
    else:
        for line in lines:
            if '#declare ' + color + ' ' in line:
                if "rgb" in line:
                    if '<' in line and '>' in line:
                        s = 'rgb <'
                        p1 = line.find(s)+len(s)
                        p2 = line.find(',', p1)
                        red = line[p1:p2]
                        
                        s = 'rgb <' + red + ', '
                        p1 = line.find(s)+len(s)
                        p2 = line.find(',', p1)
                        green = line[p1:p2]
                        
                        s = 'rgb <' + red + ', ' + green + ', '
                        p1 = line.find(s)+len(s)
                        p2 = line.find('>', p1)
                        blue = line[p1:p2]
                    else:
                        s = 'rgb '
                        p1 = line.find(s)+len(s)
                        p2 = line.find(';', p1)
                        red = line[p1:p2]
                        green = line[p1:p2]
                        blue = line[p1:p2]
                else:
                    red = line[line.find('red')+4:line.find('green')-1]
                    green = line[line.find('green')+6:line.find('blue')-1]
                    blue = line[line.find('blue')+5:line.find(';')]
                colors2rgb[color] = (float(red), float(green), float(blue))

In [ ]:
colors2rgb

# Get EM image

In [ ]:
em_cbc5_im_raw = Image.open(os.path.join('Images', 'cbp576_3D_volume.png'))

def plot_EM(ax):
        
    em_cbc5_im = em_cbc5_im_raw.transpose(Image.FLIP_LEFT_RIGHT).transpose(Image.FLIP_TOP_BOTTOM)
    em_cbc5_im = em_cbc5_im.convert("RGBA")
    datas = em_cbc5_im.getdata()
    
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)

    em_cbc5_im.putdata(newData)
    ax.imshow(em_cbc5_im, origin='lower')
    ax.set_ylim(-20, em_cbc5_im.size[1]+380)
    ax.axis('off')
    
plot_EM(ax=plt.subplot(111))

# Plot morph images

In [ ]:
def plot_morph(ax=None, cell='ON'):
    cell_im_filename = 'CBC5.png' if cell == 'ON' else 'CBC3.png'    
    cell_im = Image.open(os.path.join('Images', cell_im_filename))
    
    cell_im = cell_im.convert("RGBA")
    cell_im = cell_im.transpose(Image.FLIP_TOP_BOTTOM)
    datas = cell_im.getdata()
    
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)

    cell_im.putdata(newData)
    ax.imshow(cell_im, origin='lower')
    ax.axis('off')
    
    if cell == 'OFF': ax.set_ylim(-50, cell_im.size[1]-128)
    
plot_morph(cell='ON', ax=plt.subplot(121))
plot_morph(cell='OFF', ax=plt.subplot(122))

## Plot cable

In [ ]:
def plot_cable(ax):
    ax.imshow(Image.open(os.path.join('Images', 'cable.png')))
    ax.axis('off')
    ax.axis('equal')
    
plot_cable(ax=plt.subplot(111))

# Sanity checks

## Cone size

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(8,8))

plot_morph(ax=axs[0],cell='ON')
plot_morph(ax=axs[0],cell='OFF')

plot_EM(ax=axs[1])
plot_morph(ax=axs[1])

## EM vs morph size

In [ ]:
fig = plt.figure(1,(6,5))
ax = fig.add_axes((0,0,1,1))
plot_EM(ax=ax)
ax = fig.add_axes((0,0,0.999,1))
plot_morph(ax=ax)

In [ ]:
def plot_legend(ax):
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.patch.set_alpha(0.0)
    ax.grid(True)
    ax.axis('off')
    x_pos = 0.45
    x_len = 0.01

    y_top = 0.97
    y_bot = 0.91
    name = 'Cones'
    make_rect('Outer segment (OS)', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.86
    name = 'Cones'
    make_rect('Inner segment / Soma (IS/S)', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.77
    name = 'Cones'
    make_rect('Axon (A)', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.72
    name = 'Cones'
    make_rect('Axon Terminals (AT)', name, x_pos, y_bot, x_len, y_top)

    plt.text(x_pos-0.015, 0.72+0.5*(0.97-0.72), 'Cones', rotation=90, ha='center', va='center')
    plt.plot([x_pos-0.01, x_pos], [0.72, 0.72], c='k', lw=1)
    plt.plot([x_pos-0.01, x_pos], [0.97, 0.97], c='k', lw=1)

    y_top = y_bot
    y_bot = 0.66
    name = 'Synaptic connections'
    make_rect(name, name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.60
    name = 'Distal dendrite'
    make_rect(name + ' (DD)', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.5
    name = 'Proximal dendrite'
    make_rect(name + ' (PD)', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.45
    name = 'Soma'
    make_rect(name + ' (S)', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.25
    name = 'Proximal axon'
    make_rect(name + ' (PA)', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.14
    name = 'Distal axon'
    make_rect(name + ' (DA)', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.07
    name = 'Axon terminals'
    make_rect(name + ' (AT)', name, x_pos, y_bot, x_len, y_top)

    plt.text(x_pos-0.015, 0.07+0.5*(0.66-0.07), 'Bipolar cell', rotation=90, ha='center', va='center')
    plt.plot([x_pos-0.01, x_pos], [0.07, 0.07], c='k', lw=1)
    plt.plot([x_pos-0.01, x_pos], [0.66, 0.66], c='k', lw=1)

    y_top = y_bot
    y_bot = 0.05
    name = 'Synaptic connections'
    make_rect('', name, x_pos, y_bot, x_len, y_top)

    y_top = y_bot
    y_bot = 0.02
    name = 'Postsynaptic'
    make_rect(name, name, x_pos, y_bot, x_len, y_top)

In [ ]:
def make_rect(subname, name, x_pos, y_bot, x_len, y_top):
    rect = patches.Rectangle((x_pos,y_bot),x_len,y_top-y_bot,linewidth=1,
                         facecolor=colors2rgb[regions2color[name]], edgecolor='k', alpha=.6)
    ax.add_patch(rect)    
    ax.text(x_pos+0.15, y_bot +0.5*(y_top-y_bot), subname, va='center', ha='center')

# Make plot

In [ ]:
import matplotlib.patches as patches

In [ ]:
regions2color = {
    'Cones': 'Red',
    'Synaptic connections': 'Gray40',
    'Distal dendrite': 'Navy',
    'Proximal dendrite': 'BlueViolet',
    'Soma': 'MediumBlue',
    'Proximal axon': 'NeonBlue',
    'Distal axon': 'SummerSky',
    'Axon terminals': 'NewMidnightBlue',
    'Postsynaptic': 'Orange',
}

In [ ]:
fig = plt.figure(figsize=(5.6,4.8))

y0 = 0.36
title_ytop = 0.96-y0

ax = fig.add_axes((0.01,y0,0.25,1-y0))
plot_EM(ax=ax)
ax = fig.add_axes((0.01,y0,.9,title_ytop))
ax.set_title('A', fontweight='bold', ha='left', va='top', loc='left')
ax.axis('off')

morph_ON_x = 0.2
ax = fig.add_axes((morph_ON_x,y0,0.3,1-y0))
plot_morph(ax=ax, cell='ON')
ax = fig.add_axes((morph_ON_x+0.03,y0,.9,title_ytop))
ax.set_title('B i', fontweight='bold', ha='left', va='top', loc='left')
ax.axis('off')

morph_OFF_x = 0.73
ax = fig.add_axes((morph_OFF_x,y0,0.3,1-y0))
plot_morph(ax=ax, cell='OFF')
ax = fig.add_axes((morph_OFF_x+0.03,y0,.95,title_ytop))
ax.set_title('B ii', fontweight='bold', ha='left', va='top', loc='left')
ax.axis('off')

ax = fig.add_axes((0.01,y0,1,1-y0))
plot_legend(ax)

ax = fig.add_axes((0,0,1,y0*0.95))
plot_cable(ax=ax)
ax = fig.add_axes((0,y0-0.15,.9,0.1))
ax.set_title('C', fontweight='bold', ha='left', va='top', loc='left')
ax.axis('off')

data_utils.make_dir('Figures')
plt.savefig(f'../_figures/fig{fig_num}_morph2cbcpluscable.pdf', dpi=600)
plt.savefig(os.path.join('Figures', 'morph2cbc')+'.png', dpi=100)
plt.show()

In [ ]:
# Good?
plt.imshow(Image.open(os.path.join('Figures', 'morph2cbc')+'.png'))